In [1]:
import pandas as pd

csv = pd.read_csv('index_label_to_category.csv')
# csv.set_index('landmark_id')
csv = csv[9000:10000]
csv

print(len(csv))

101302


In [2]:
country_map = pd.read_csv('countries_codes_and_coordinates.csv')
country_map


,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,"""AF""","""AFG""","""4""","""33""","""65"""
1,Albania,"""AL""","""ALB""","""8""","""41""","""20"""
2,Algeria,"""DZ""","""DZA""","""12""","""28""","""3"""
3,American Samoa,"""AS""","""ASM""","""16""","""-14.3333""","""-170"""
4,Andorra,"""AD""","""AND""","""20""","""42.5""","""1.6"""
...,...,...,...,...,...,...
251,Wallis and Futuna,"""WF""","""WLF""","""876""","""-13.3""","""-176.2"""
252,Western Sahara,"""EH""","""ESH""","""732""","""24.5""","""-13"""
253,Yemen,"""YE""","""YEM""","""887""","""15""","""48"""
254,Zambia,"""ZM""","""ZMB""","""894""","""-15""","""30"""


In [3]:
#test for country code
# url = 'https://commons.wikimedia.org/wiki/Category:Seal_Bay_Conservation_Park'
def countryCodeFromURL (url):
    table = soup.find('tbody')
    loc = [t.parent for t in table.findAll(text='Location')]
    row = [t.parent for t in loc]
    country = row[0].text.split(',')[-1].strip()
    country = country.split(' ')[-1].strip()
    twoCode = country_map.loc[country_map['Country'] == country]
    twoCode = twoCode['Alpha-2 code'].tolist()[0]
    twoCode = twoCode[2:-1]
    return twoCode
# twoCode

In [4]:
from bs4 import BeautifulSoup
import requests

def nameFromURL (soup):
    landmark_name = soup.find(id = 'wdinfoboxcaption')
    landmark_name = landmark_name.text
    return landmark_name
# print(landmark_name)

In [5]:
def latLongFromURL (soup):
    allLinks = soup.find(id="bodyContent").find_all("a")
    linkToScrape = 0

    for link in allLinks:
        if 'geohack' in link['href']: 
            linkToScrape = 'https:' + link['href']

    latLongDFs = pd.read_html(linkToScrape)
    latLong = latLongDFs[0][1][0].split('E')[1]
    return latLong
# print(latLong)

In [6]:
def imageFromURL (soup):
    image = soup.find('img')['src']
    suffix = image.split('/')[-1][3:]
    suffix = '500' + suffix
    prefix = image[:image.rindex('/')+1]
    image = prefix + suffix
    return image

In [7]:
pd.options.mode.chained_assignment = None  # default='warn'
csv['landmark_name'] = 0
csv['image_url'] = 0
csv['country_code'] = None
csv['latitude'] = None
csv['longitude'] = None
i = 0 
for (index, row) in csv.iterrows():
    url = row['category']
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    try:
        csv['landmark_name'][index] = nameFromURL(soup)
    except:
        continue
    try:
        csv['image_url'][index] = imageFromURL(soup)
    except:
        print(row['category'])
        csv['image_url'][index] = None
    try:
        csv['country_code'][index] = countryCodeFromURL(soup)
    except:
        continue
    try:

        lat_long = str(latLongFromURL(soup)).split(',')
        csv['latitude'][index] = lat_long[0]
        csv['longitude'][index] = lat_long[1]
    except:
        continue
    i = i + 1
    if (i % 100 == 0): 
        out = csv.dropna(subset = ['landmark_name', 'latitude', 'longitude', 'country_code'])
        out.to_csv('scrape/landmarks.csv'.format(index))

KeyboardInterrupt: 

,landmark_id,category,landmark_name,image_url,country_code,latitude,longitude
0,0,http://commons.wikimedia.org/wiki/Category:Sea...,Seal Bay Conservation Park,https://upload.wikimedia.org/wikipedia/commons...,None,None,None
1,1,http://commons.wikimedia.org/wiki/Category:Fra...,Building at 10 Follen Street,https://upload.wikimedia.org/wikipedia/commons...,None,None,None
2,2,http://commons.wikimedia.org/wiki/Category:Rin...,Ringsjön,https://upload.wikimedia.org/wikipedia/commons...,SE,55.872222,13.518611
3,3,http://commons.wikimedia.org/wiki/Category:Tak...,Takeda Shrine,https://upload.wikimedia.org/wikipedia/commons...,JP,35.686667,138.5775
4,4,http://commons.wikimedia.org/wiki/Category:Iep...,Ieperlee,https://upload.wikimedia.org/wikipedia/commons...,BE,50.9833,2.8
5,5,http://commons.wikimedia.org/wiki/Category:Puf...,Puffing Billy Railway,https://upload.wikimedia.org/wikipedia/commons...,None,None,None
6,6,http://commons.wikimedia.org/wiki/Category:Lak...,Lake of the Clouds,https://upload.wikimedia.org/wikipedia/commons...,None,None,None
7,7,http://commons.wikimedia.org/wiki/Category:Ike...,Ikema Island,https://upload.wikimedia.org/wikipedia/commons...,JP,24.932222,125.241944
8,8,http://commons.wikimedia.org/wiki/Category:Pet...,Petersfriedhof,https://upload.wikimedia.org/wikipedia/commons...,AT,47.796667,13.045556
9,9,http://commons.wikimedia.org/wiki/Category:New...,New Valamo,https://upload.wikimedia.org/wikipedia/commons...,FI,62.563056,28.790556


In [ ]:
csv[]

SyntaxError: invalid syntax (<ipython-input-134-c7593165d623>, line 1)